In [6]:
import base64
import requests
import time

def transcribe(audioStr):
    try:
        messages = [
            {
                "role": "system",
                "content": "You are responsible for transcribing audio recordings into text. The application context is: Taiwan Congress."
            },
            {
                "role": "user",
                "content": [
                    { "type": "audio_url", "audio_url": { "url": audioStr } },
                    { "type": "text", "text": "提取文本" }
                ]
            }
        ]

        # 请将以下 URL 替换为实际的 API 端点
        url = 'https://mtt0f4kcgk6iw5-5001.proxy.runpod.net/speech2text'

        headers = {
            'Content-Type': 'application/json'
            # 如需身份验证，请在此处添加 'Authorization' 字段
        }

        payload = {
            'model': 'Training/wsasr/v18-20241104-130300/checkpoint-500-merged',
            'max_tokens': 256,
            'temperature': 0.1,
            'messages': messages
        }

        response = requests.post(url, headers=headers, json=payload)

        if not response.ok:
            raise Exception(f"HTTP error! status: {response.status_code}")

        data = response.json()

        return data

    except Exception as error:
        print('Error:', error)
        raise error

audio_path = 'demo/sample.mp3'

# 读取 MP3 文件的二进制数据
with open(audio_path, 'rb') as file:
        binary_data = file.read()
# 将二进制数据转换为 base64 编码的字符串
base64_encoded_data = base64.b64encode(binary_data).decode('utf-8')
data_uri = f'data:audio/mp3;base64,{base64_encoded_data}'
start_time = time.time()
segments = transcribe(data_uri)
print('Time:', time.time() - start_time)
print(segments)

Time: 5.171081066131592
{'segments': [{'start': 0.1, 'end': 11.82, 'words': [{'start': 0.1, 'end': 0.12, 'word': '好', 'confidence': 0.80615234375}, {'start': 0.24, 'end': 0.26, 'word': '被', 'confidence': 0.7646484375}, {'start': 0.38, 'end': 0.4, 'word': '撞', 'confidence': 0.68115234375}, {'start': 0.62, 'end': 0.64, 'word': '擊', 'confidence': 0.0}, {'start': 0.7000000000000001, 'end': 0.72, 'word': '而', 'confidence': 0.12420654296875}, {'start': 0.8, 'end': 0.8200000000000001, 'word': '不', 'confidence': 0.98779296875}, {'start': 0.9400000000000001, 'end': 0.96, 'word': '幸', 'confidence': 0.00015354156494140625}, {'start': 1.1400000000000001, 'end': 1.16, 'word': '身', 'confidence': 0.96142578125}, {'start': 1.28, 'end': 1.3, 'word': '亡', 'confidence': 0.0179595947265625}, {'start': 1.4000000000000001, 'end': 1.42, 'word': '的', 'confidence': 0.8857421875}, {'start': 1.52, 'end': 1.54, 'word': '蘇', 'confidence': 0.0}, {'start': 1.74, 'end': 1.76, 'word': '姓', 'confidence': 4.136562347412

In [ ]:
import re
import time

def remove_repeated_phrases(text):
    # 移除句子末尾的空白字符
    text = text.rstrip()

    max_phrase_length = 30  # 最大短语长度，可根据需要调整
    min_repeats = 5  # 最小重复次数

    # 获取文本长度
    text_length = len(text)

    # 计算可能的最大短语长度，防止越界
    max_possible_length = min(max_phrase_length, text_length // min_repeats)

    # 从短语长度为 1 开始，逐渐增加
    for phrase_length in range(1, max_possible_length + 1):
        # 获取末尾的短语
        phrase = text[-phrase_length:]
        # 构建重复的短语
        repeated_phrase = phrase * min_repeats
        # 检查文本是否以重复的短语结尾
        if text.endswith(repeated_phrase):
            # 使用正则表达式查找重复的部分
            pattern = f'({re.escape(phrase)}){{{min_repeats},}}$'
            match = re.search(pattern, text)
            if match:
                # 将重复的部分替换为单个短语
                text = text[:match.start()] + ''
                ntlen = len(text)
                plen = len(phrase)
                index = None
                for i in range(0, plen):
                    if(text[ntlen-i-1] == phrase[plen-i-1]):
                        index = i
                        continue
                    else:
                        break
                if(index != None):
                    text = text+phrase[:-index-1]
                return text

    # 如果未找到重复的短语，返回原文本
    return text

# 示例文本
texts = [
    "和蓉。是您來關注這起嫌犯開贓車撞死巡路員的的的的...的的的...的的的...的的的...的的的...的的的...的的的...的的的...的的的...的的的...的的的...的的的...的的的...的的的...的的的...的的的...的的的...的的的...的的的...的的的...的的的...的的的...的的的...的的的...的的的...的的的",
    "而同一時間掀掀的掀掀的掀掀的掀掀的掀掀的掀掀的掀掀的掀掀的掀掀的掀掀",
    "悲劇事件就在今天下午的就在今天下午的就在今天下午的就在今天下午的就在今天下午的就在今天下午的就在今天下午的就在今天下午的就在今天下午的就在今天下午的就在今天下午的就在今天下午的就在今天下午的就在今天下午的就在今天下午的就在今天下午的就在今天下午的就在今天下午的就在今天下午的就在今天下午的就在今天下午的就在今天下午的就在今天下午的就在今天下午的就在今天下午的就在今天下午的就在今天下午的就在今天下午的就在今天下午的就在今天下午的就在今天下午的就在"
]

# 测试代码
for idx, text in enumerate(texts, 1):
    start_time = time.time()
    result = remove_repeated_phrases(text)
    print('Time:', time.time() - start_time)
    print(f"示例 {idx}：\n原文本：{text}\n处理后：{result}\n")

Time: 6.580352783203125e-05
示例 1：
原文本：和蓉。是您來關注這起嫌犯開贓車撞死巡路員的的的的...的的的...的的的...的的的...的的的...的的的...的的的...的的的...的的的...的的的...的的的...的的的...的的的...的的的...的的的...的的的...的的的...的的的...的的的...的的的...的的的...的的的...的的的...的的的...的的的...的的的
处理后：和蓉。是您來關注這起嫌犯開贓車撞死巡路員的的的的...

Time: 2.7894973754882812e-05
示例 2：
原文本：而同一時間掀掀的掀掀的掀掀的掀掀的掀掀的掀掀的掀掀的掀掀的掀掀的掀掀
处理后：而同一時間掀掀的

Time: 1.1920928955078125e-05
示例 3：
原文本：悲劇事件就在今天下午的就在今天下午的就在今天下午的就在今天下午的就在今天下午的就在今天下午的就在今天下午的就在今天下午的就在今天下午的就在今天下午的就在今天下午的就在今天下午的就在今天下午的就在今天下午的就在今天下午的就在今天下午的就在今天下午的就在今天下午的就在今天下午的就在今天下午的就在今天下午的就在今天下午的就在今天下午的就在今天下午的就在今天下午的就在今天下午的就在今天下午的就在今天下午的就在今天下午的就在今天下午的就在今天下午的就在
处理后：悲劇事件就在今天下午的



In [27]:
import re

pattern = r'(?:\d+(?:,{1,2}\d+)?\s*->\s*)+\d+(?:,{1,2}\d+)?'
text = '把你0,000 -> 00,673 配給所有台灣人的感受的00,723 -> 04,,008 基本檢驗到了的時候'

text = re.sub(pattern, r'', text)
print(text)

把你 配給所有台灣人的感受的 基本檢驗到了的時候
